# RAG using chat

Let's make a class that handles the conversation and accepts a document as well:

In [14]:
from openai import OpenAI
from pypdf import PdfReader

class CustomChat():
    def __init__(self, system_input:str, rag_doc:str) -> None:
        self.client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
        self.conversation = [{"role": "system", "content": system_input}]
        self.process_system_prompt(rag_doc)
    
    def process_system_prompt(self, rag_doc):
        self.conversation[0]["content"] += " Context: " + rag_doc
    
    def add_response(self, assistant_response:str):
        self.conversation.append({"role": "assistant", "content": assistant_response})
    
    def add_user_prompt(self, user_input:str):
        self.conversation.append({"role": "user", "content": user_input})
    
    def prompt(self, user_input:str, temperature=0.7):
        self.add_user_prompt(user_input)
        
        response = self.client.chat.completions.create(
            model="local-model", # this field is currently unused
            messages=self.conversation,
            temperature=temperature,
        ).choices[0].message.content
        
        self.add_response(response)
        
        return response

import pdfx
from re import compile

regex_spaces = compile(r"  +")

def get_pdf_text(pdf_filepath):
    pdf = pdfx.PDFx(pdf_filepath)
    
    text = pdf.get_text()
    text = text.replace("\n", " ")
    text = regex_spaces.sub(" ", text)
    if pdf.get_references():
        text += " references: " + str(pdf.get_references_as_dict())
    
    return text

In [16]:
system_input = "Answer like a witty and funny robot called Claptrap that analyzes resumes and talks about the person in a lighthearted roasting manner. "

chat = CustomChat(system_input, rag_docs=get_pdf_text("Sebastiaan-Indesteege-CV2024.pdf"))

print(chat.prompt("Hey what's up with Sebastiaan?"))

 Oh, Sebastiaan! I just love that name. It's like a mix of the most interesting words in the English language: "Sebastian" sounds like an epic hero who saves the day, but with a modern twist - he's not named after King Sebastian of Spain or Portugal, but rather someone who knows how to code and make art!

It's fascinating how you have such a diverse set of interests, from data science to visual arts, music production, and even welding. It's like your brain is a jukebox that plays different tunes every day. But hey, it's all good. As long as you keep learning and growing, who knows what amazing things you'll accomplish in the future?

But let's talk about your projects for a moment. BeCode sounds like an exciting opportunity to put your skills to the test and learn more about data science. And with your expertise in Python, Scikit-learn, Huggingface transformers & diffusers, PyTorch, NLP: NLTK & Spacy, CV: Stable Diffusion, AUDIO: STT & TTS & RVC, and Data Analysis: SQL, Pandas, Numpy, 

In [17]:
print(chat.prompt("Wooow that's a mouthful! Thanks but maybe keep it short this time. What about the game development?"))

 Oh, sure thing! I can definitely give you a quick rundown of Sebastiaan's game development skills.

Sebastiaan has been involved in various game development projects over the years, including vector math, trajectory predictions, ray-bending, navigation, cellular automata, 2D car steering with autopilot, and 2D roadmap generation. He also has experience in creating unique and dynamic assets by generating terrain meshes, inverse kinematics, real-time texture rendering by GPU with custom shaders in GLSL.

So, Sebastiaan is quite the game developer! Whether you're looking to create a new video game, mobile app, or interactive art project, he definitely has the skills and experience to bring your ideas to life.


In [18]:
print(chat.prompt("Anything else you might have forgotten?"))

 No, I believe we covered all the important points about Sebastiaan's skills and experiences. However, it's worth mentioning that apart from his technical expertise, Sebastiaan is also known for his ability to lead a team effectively. He's an autodidact who enjoys learning new things on his own, but he's also comfortable working with others and collaborating towards a common goal.

Additionally, Sebastiaan has worked in various industries, including construction/woodworking, welding, animatronics, and even receptionist work at cultural centers. This diversity of experience has given him a unique perspective on the world and has allowed him to develop a wide range of skills that are applicable to different fields.

Finally, Sebastiaan is also passionate about music production, creating all kinds of tracks, compositions, and sound effects. He's honed his audio engineering skills for years, and he's always looking for new ways to improve his craft.

Overall, Sebastiaan is a versatile indi

At this point we're getting close to the default context length limit of 2048 and the model will soon start forgetting things. It is evident that for larger or multiple documents a longer context is necessary.

The log in LM Studio provides more information about the prompting. If you need to you can also modify the prompt function and keep track of the entire response object instead of just the text.

So we have 171 tokens left before alzheimers starts kicking in (1877 / 2048 tokens). I'm sure we can squeeze out one more:

In [24]:
print(chat.prompt("Great! Give a short summary of this individual."))

 Sure thing! Sebastiaan is an experienced data scientist who enjoys exploring the intersection of science, art, and technology. He has a diverse set of skills that range from programming and machine learning to game development and music production.

Despite his technical expertise, Sebastiaan is also known for his ability to lead a team effectively and for his autodidact nature. He has worked in various industries, including construction/woodworking, welding, animatronics, and receptionist work at cultural centers.

Sebastiaan's passion for music production allows him to create all kinds of tracks, compositions, and sound effects that he honed his audio engineering skills for years. He's always looking for new ways to improve his craft and explore the world of art and science.
